In [ ]:
# PySpark transformations: Bronze → Silver

from pyspark.sql.functions import col, to_timestamp

# Paths
bronze_path = "abfss://bronze@energydatalaketest.dfs.core.windows.net/load/"
silver_path = "abfss://silver@energydatalaketest.dfs.core.windows.net/load/"

# Read raw JSON from Bronze
df_raw = spark.read.json(bronze_path)

# Explode + Clean schema
df_silver = (
    df_raw.selectExpr("explode(response_data) AS data")
          .select(
              to_timestamp(col("data.period")).alias("ts"),
              col("data.respondent").alias("region"),
              col("data.`type`").alias("type"),
              col("data.`type-name`").alias("type_name"),
              col("data.value").cast("double").alias("mw"),
              col("data.`value-units`").alias("unit")
          )
          .dropna(subset=["ts", "mw"])
)

# Save as Delta to Silver
df_silver.write.format("delta").mode("overwrite").save(silver_path)
